In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import sklearn
import pandas as pd
import numpy as np

from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm, trange,tnrange,tqdm_notebook
from transformers import BertTokenizer, BertConfig
import random
import shutil

/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load Data 

In [2]:
import string
punctuation = string.punctuation
import nltk

import re
def sent_tokenize(df,sent_method):
    sents = []
    for index,row in df.iterrows():
        sent = []
        tempSent = row['utterance'].split('|||')
        if sent_method == 'nltk':
            for i in tempSent:
                sent = sent + nltk.sent_tokenize(i)
        elif sent_method == 'comma':
            for i in tempSent:
                sent = sent + [j.strip().lower() for j in re.split(r"[,,.,?,!,;()]",i) if len(j.strip().split(' '))>1]
        sents = sents + sent
    return sents


data_paths = [
    '../data/FriendsPersona/Friends_A_whole.tsv',
    '../data/myPersonality/MyPersonality_A_whole.tsv',
    '../data/pan2015/Pan_A_whole.tsv',
    '../data/Essay/Essay_A_whole.tsv',
#     '../data/Kaggle_mbti/Kaggle_map_label_words_comma.tsv'  
]

df_data = pd.DataFrame([])
for path in data_paths:
    df = pd.read_csv(path,  sep='\t')
    sents = sent_tokenize(df,'comma')
    df_sents = pd.DataFrame(sents)
    df_data = pd.concat([df_data, df_sents], axis=0)
    print(df_data.shape)
df_data = df_data.drop_duplicates()
print(df_data.shape)

(5631, 1)
(24140, 1)
(54665, 1)
(213084, 1)
(200015, 1)


In [3]:
def get_span(pos, row):
    if pos == 1:
        return row['sent'].split(row['label_word'])[0]
    elif pos == 2:
        try:
            return row['sent'].split(row['label_word'])[1]
        except:
            return ""

## Construct Training Samples

In [4]:

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

def get_input_and_target(df):
    span_1   = '<extra_id_0> '
    span_2   = ' <extra_id_1> '
    span_end = ' <extra_id_2>'
    tmp_df = df.fillna(" ")
    tmp_df['inputs'] = span_1 + tmp_df['label_word'] + span_2
    tmp_df['target'] = span_1 + tmp_df['span_1'] + span_2 + tmp_df['span_2'] + span_end
    return list(tmp_df['inputs']), list(tmp_df['target'])



def construct_training(df_data_, tokenizer):
    inputs, targets = get_input_and_target(df_data_)
    max_len    = 20
    batch_size = 16

    inputs  = [tokenizer.encode(sent, add_special_tokens=True, max_length=max_len, pad_to_max_length=True) for sent in inputs]
    targets = [tokenizer.encode(sent, add_special_tokens=True, max_length=max_len, pad_to_max_length=True) for sent in targets]

    input_attention_masks = [[float(i>0) for i in seq] for seq in inputs]



    train_inputs  = inputs
    train_masks   = input_attention_masks
    train_targets = targets


    train_inputs     = torch.tensor(train_inputs)
    train_targets    = torch.tensor(train_targets)
    train_masks      = torch.tensor(train_masks)

    train_data       = TensorDataset(train_inputs, train_masks, train_targets)
    train_sampler    = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    return train_dataloader


In [5]:
import datetime
label_words = []
for trait in ['A', 'C','E','O','N']:
    print('Processing trait ', trait, '...')
    label_words = []
    with open('label_words/'+trait+'_words.txt', 'r') as f:
        pos = f.readline().split(',')
        neg = f.readline().split(',')
    label_words += pos
    label_words += neg
    
    label_words = set(label_words)
    
    refined_label_words = label_words - set(('think', 'about', 'really', 'work', 'mean', 'kind'))
    
    starttime = datetime.datetime.now()
    context_dict = {}
    label_word_dict = {}

    sent_with_label_word = []
    for sent in df_data[0]: 
        for word in refined_label_words: 
            if word in sent.split(' '):
                try:
                    label_word_dict[word] += 1
                except:
                    label_word_dict[word] = 1
                sent_with_label_word.append([sent, word])
                w_list = sent.split(' ')
                for w in w_list:
                    if not w == word:
                        try:
                            context_dict[w] += 1
                        except:
                            context_dict[w] = 1
    
    
    label_word_dict = {k: v for k, v in sorted(label_word_dict.items(), key=lambda item: abs(item[1]), reverse=True)}
    print('Number of matched label words: ', len(label_word_dict))
    print('Top 10 matched label words: ', list(label_word_dict.items())[:10])
    
    print('Number of sents containing label words: ', len(sent_with_label_word))
    print('Top 10 sents containing label words: ', sent_with_label_word[:10])
    
    
    
    df_data_ = pd.DataFrame([])
    df_data_['sent'] = [i[0] for i in sent_with_label_word]
    df_data_['label_word'] = [i[1] for i in sent_with_label_word]
    
    df_data_['span_1'] = df_data_.apply(lambda x: get_span(1, x), axis=1)
    df_data_['span_2'] = df_data_.apply(lambda x: get_span(2, x), axis=1)
    
    
    tokenizer = T5Tokenizer.from_pretrained("t5-large")
    model = T5ForConditionalGeneration.from_pretrained("t5-large")
    
    print('Construct Training Samples...')
    train_dataloader = construct_training(df_data_, tokenizer)
    
    print('Second phase Pre-training...')    
    
    
    num_epoch          = 20
    learning_rate      = 1e-4
    adam_epsilon       = 1e-8
    num_warmup_steps   = 0
    num_training_steps = len(train_dataloader)*num_epoch


    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr = learning_rate, eps = adam_epsilon, correct_bias = False)  
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  


    model.cuda()
    for _ in tnrange(1, num_epoch+1, desc='Epoch'):
        print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
        batch_loss = 0
        for step, batch in enumerate(train_dataloader):
            model.train()
            batch = tuple(t.cuda() for t in batch)
            b_input_ids, b_input_masks, b_labels = batch

            loss = model(input_ids=b_input_ids, attention_mask=b_input_masks, labels=b_labels).loss

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

            batch_loss += loss.item()

        print(batch_loss)
    
    model_path = 'Adapted_t5_large_'+trait+'/'
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)
    
    endtime = datetime.datetime.now()
    print((endtime - starttime))
    print()
    

Processing trait  A ...
Number of matched label words:  654
Top 10 matched label words:  [('great', 1553), ('happy', 1276), ('nice', 1207), ('able', 998), ('funny', 779), ('reason', 706), ('cold', 516), ('easy', 504), ('free', 501), ('ready', 463)]
Number of sents containing label words:  23979
Top 10 sents containing label words:  [["i'm nervous", 'nervous'], ['you look so young', 'young'], ['very nice touch', 'nice'], ["these'll go great in my new place", 'great'], ['based on serious stuff', 'serious'], ["i'd be able to be a stand-up guy and go the distance", 'able'], ["i don't mean to be disrespectful", 'disrespectful'], ['this is great', 'great'], ['very sexy', 'sexy'], ['joey gets her something really great', 'great']]


/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you enc

Construct Training Samples...


/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Second phase Pre-training...


...

/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Second phase Pre-training...


/home/wenzhy/wenzhy/ENTER/envs/wen/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_141613/4087040875.py:76: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for _ in tnrange(1, num_epoch+1, desc='Epoch'):


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>
3186.9186861515045
<====================== Epoch 2 ======================>
2754.942104101181
<====================== Epoch 3 ======================>
2507.4792774915695
<====================== Epoch 4 ======================>
2293.4754366874695
<====================== Epoch 5 ======================>
2111.269564270973
<====================== Epoch 6 ======================>
1951.4770336151123
<====================== Epoch 7 ======================>
1807.9176329374313
<====================== Epoch 8 ======================>
1679.0132877230644
<====================== Epoch 9 ======================>
1563.3644071221352
<====================== Epoch 10 ======================>
1457.059626340866
<====================== Epoch 11 ======================>
1363.3138824105263
<====================== Epoch 12 ======================>
1279.610775232315
<====================== Epoch 13 ======================>
1199.4739170074463
<====================== E